Richard Albright

ISYE65501


Question 15.2

In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem with real data. The data is given in the file diet.xls.

Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using PuLP. Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)
  

Import the pulp and pandas module that are needed for this problem.

In [1]:
from pulp import *
import pandas as pd
import warnings
warnings.simplefilter('ignore')

Use pandas to read in the excel file

In [2]:
excel = pd.read_excel('diet.xls')

Since the excel file has 2 sections in the sheet, create 2 dataframes, one containing the diet, and the other containing the constraints.

In [3]:
diet = excel[:64]
constraints = excel[65:]
constraints.drop(['Foods', 'Price/ Serving'], axis=1, inplace=True)

Display both of our data frames to ensure they loaded properly.

In [4]:
diet.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [5]:
constraints

,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
65,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


Convert the nutrients constraints data frame to nmix and nmax dictionaries.

In [6]:
nutrients= list(constraints.columns.values)[1:]
nmin = dict(zip(nutrients, constraints.values.tolist()[0][1:]))
nmax = dict(zip(nutrients, constraints.values.tolist()[1][1:]))

Convert the diet dataframe to list and create a separate list of their names.

In [7]:
foods = diet.values.tolist()
foodNames = [x[0] for x in foods]

Create a cost dictionary which contains the cost of each food.

In [8]:
cost = dict([(x[0], x[1]) for x in foods])

Create a dictionary of nutrients containing the nutrients of each food.

In [9]:
nutrient = {}
for i in range(len(nutrients)):
    nutr = nutrients[i]
    nutrient[nutr] = dict([(f[0], f[i+3]) for f in foods])

Create the optimization problem using the pulps moddule, minimizing the cost of food while meeting the nutritional constaints.

In [10]:
prob = LpProblem("The Diet Problem", LpMinimize)

Assign the objective function to minimize the cost to the problem.

In [11]:
food_vars = LpVariable.dicts("Food",foodNames,0)
prob += lpSum([cost[food]*food_vars[food] for food in foodNames])
food_selected_vars = LpVariable.dicts("Selected", foodNames, lowBound=0, upBound=1, cat="Binary")

Assign the food variables with their nutritional constraints to the problem.

In [12]:
for n in nutrients:
    prob += lpSum([nutrient[n][f] * food_vars[f] for f in foodNames]) >= nmin[n]
    prob += lpSum([nutrient[n][f] * food_vars[f] for f in foodNames]) <= nmax[n]


Solve the problem.

In [13]:
prob.solve()

1

Display the solution.

In [14]:
for var in prob.variables():
    if var.varValue > 0:
        print(str(var.varValue)+" units of "+str(var).replace('Food_','') )
print()
print("Total cost of food = $%.2f" % value(prob.objective)) 

52.64371 units of Celery,_Raw
0.25960653 units of Frozen_Broccoli
63.988506 units of Lettuce,Iceberg,Raw
2.2929389 units of Oranges
0.14184397 units of Poached_Eggs
13.869322 units of Popcorn,Air_Popped

Total cost of food = $4.34



Please add to your model the following constraints (which might require adding more variables) and solve the new model:

a. If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will
need two variables for each food i: whether it is chosen, and how much is part of the diet.
You’ll also need to write a constraint to link them.)

Recreate the problem above but add a 2nd binary value to store if the food was selected, so we can use it for our additional constraints.

In [15]:
prob2 = LpProblem("The Diet Problem", LpMinimize)
food_vars = LpVariable.dicts("Food",foodNames,0)
prob2 += lpSum([cost[food]*food_vars[food] for food in foodNames])
food_selected_vars = LpVariable.dicts("Selected", foodNames, lowBound=0, upBound=1, cat="Binary")

for n in nutrients:
    prob2 += lpSum([nutrient[n][f] * food_vars[f] for f in foodNames]) >= nmin[n]
    prob2+= lpSum([nutrient[n][f] * food_vars[f] for f in foodNames]) <= nmax[n]

Add the constraint of selecting at least 1/10 serving.

In [16]:
for f in foodNames:
    prob2 += food_vars[f] <= food_selected_vars[f] * 1000000
    prob2 += food_vars[f] >= food_selected_vars[f] * 0.1

b. Many people dislike celery and frozen broccoli. So at most one, but not both, can be
selected.

Allow at most one celery of broccoli to be selected.

In [17]:
 prob2 += food_selected_vars['Celery, Raw'] + food_selected_vars['Frozen Broccoli'] <= 1

c. To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be
selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered meat?), just call it whatever you think is appropriate – I want you to learn how to write this type of constraint, but I don’t really care whether we agree on how to classify foods!]

At least 3 kinds of meat/poultry/fish/eggs must be selected.

In [18]:
meats = [
    'Roasted Chicken',
    'Poached Eggs',
    'Scrambled Eggs',
    'Bologna,Turkey',
    'Frankfurter, Beef',
    'Ham,Sliced,Extralean',
    'Kielbasa,Prk',
    'Taco',
    'Hamburger W/Toppings',
    'Hotdog, Plain','Pork',
    'Sardines in Oil',
    'White Tuna in Water']

prob2 += lpSum([food_selected_vars[meat] for meat in meats]) >= 3

Solve the optimization problem.

In [19]:
prob2.solve()

1

Display the solution.

In [20]:
for var in prob2.variables():
    if var.varValue > 0.0:
        print(str(var.varValue)+" units of "+str(var).replace('Food_','') )
print()
print("Total cost of food = $%.2f" % value(prob2.objective)) 

42.399358 units of Celery,_Raw
0.1 units of Kielbasa,Prk
82.802586 units of Lettuce,Iceberg,Raw
3.0771841 units of Oranges
1.9429716 units of Peanut_Butter
0.1 units of Poached_Eggs
13.223294 units of Popcorn,Air_Popped
0.1 units of Scrambled_Eggs
1.0 units of Selected_Celery,_Raw
1.0 units of Selected_Kielbasa,Prk
1.0 units of Selected_Lettuce,Iceberg,Raw
1.0 units of Selected_Oranges
1.0 units of Selected_Peanut_Butter
1.0 units of Selected_Poached_Eggs
1.0 units of Selected_Popcorn,Air_Popped
1.0 units of Selected_Scrambled_Eggs

Total cost of food = $4.51


If you want to see what a more full-sized problem would look like, try solving your models for the file diet_large.xls, which is a low-cholesterol diet model (rather than minimizing cost, the goal is to minimize cholesterol intake). I don’t know anyone who’d want to eat this diet – the optimal solution includes dried chrysanthemum garland, raw beluga whale flipper, freeze-dried parsley, etc. – which shows why it’s necessary to add additional constraints beyond the basic ones we saw in the video!
[Note: there are many optimal solutions, all with zero cholesterol, so you might get a different one. It probably won’t be much more appetizing than mine.]

Read in the diet large excel file.

In [21]:
excel2 = pd.read_excel('diet_large.xls')

The excel file as is has no values when there should be 0s, we need to fillna the pandas dataframe to cope.

In [22]:
excel2 = excel2.fillna(0)

Check the tail of the excel2 dataframe to find our nutritional constraints.

In [23]:
excel2.tail()

,Long_Desc,Protein,"Carbohydrate, by difference",Energy,Water,Energy.1,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
7145,"Turtle, green, raw",19.8,0,89,78.5,372.0,118,1.4,20,180,...,0.15,1.1,0,0.12,15,1,0.1,50.0,0.0,0.127
7146,0,0,0,0,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0.000
7147,0,56,130,2400,3700,2400.0,1000,8,270,700,...,1.3,16,5,1.3,400,2.4,120,0.0,0.0,0.000
7148,0,g/d,g/d,kcal,g,0.0,mg/d,mg/d,mg/d,mg/d,...,mg/d,mg/d,mg/d,mg/d,microg/d,microg/d,microg/d,0.0,0.0,0.000
7149,0,1000000,1000000,1000000,1000000,1000000.0,2500,45,400,4000,...,1000000,35,1000000,100,1000,1000000,1000000,0.0,0.0,0.000


Split the datframe into 2, one including the diet, the other including the constraints.

In [24]:
diet2 = excel2[:7146]
constraints2 = excel2[7147:]
constraints2 = constraints2.drop(constraints2.index[1])
constaints2 = constraints2.fillna(0)
constraints2.drop(['Long_Desc'], axis=1, inplace=True)
constraints2

,Protein,"Carbohydrate, by difference",Energy,Water,Energy.1,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P","Potassium, K",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
7147,56,130,2400,3700,2400.0,1000,8,270,700,4700,...,1.3,16,5,1.3,400,2.4,120,0.0,0.0,0.0
7149,1000000,1000000,1000000,1000000,1000000.0,2500,45,400,4000,1000000,...,1000000,35,1000000,100,1000,1000000,1000000,0.0,0.0,0.0


Create a min and max dictionary of our nutritional constraints.

In [25]:
nutrients2 = list(constraints2.columns.values)
nmin2 = dict(zip(nutrients2, constraints2.values.tolist()[0]))
nmax2 = dict(zip(nutrients2, constraints2.values.tolist()[1]))

Create an array of all of the food names in the diet data frame.

In [26]:
foods2 = diet2.values.tolist()
foodNames2 = [x[0] for x in foods2]

Create a dictionary of cholesterol values for all items in our diet data frame.

In [27]:
cholesterol = dict([(x[0], x[28]) for x in foods2])

Create a dictionary of all other nutrients for all items in our diet data frame.

In [28]:
nutrient2 = {}
for i in range(len(nutrients2)):
    nutr = nutrients2[i]
    nutrient2[nutr] = dict([(f[0], f[i+1]) for f in foods2])

Create the optimization problem using the pulps moddule, minimizing the cholesterol intake while meeting the rest of the nutritional constaints.

In [29]:
prob3 = LpProblem("The Large Diet Problem", LpMinimize)
food_vars2 = LpVariable.dicts("Food2",foodNames2,0)
prob3 += lpSum([cholesterol[food]*food_vars2[food] for food in foodNames2])
for n in nutrients2:
    prob3 += lpSum([nutrient2[n][f] * food_vars2[f] for f in foodNames2]) >= nmin2[n]
    prob3 += lpSum([nutrient2[n][f] * food_vars2[f] for f in foodNames2]) <= nmax2[n]

Solve the problemm.

In [30]:
prob3.solve()

1

Display the results.

In [31]:
for var in prob3.variables():
    if var.varValue > 0.0:
        print(str(var.varValue)+" units of "+str(var).replace('Food_','') )
print()
print("Total cholesterol of food = %.2f" % value(prob3.objective)) 

1.1642403 units of Food2_Cereals_ready_to_eat,_NATURE'S_PATH,_OPTIMUM_SLIM
0.7518797 units of Food2_Cereals_ready_to_eat,_composite_character_cereals_(movies,_TV),
0.57206552 units of Food2_Chiton,_leathery,_gumboots_(Alaska_Native)
0.21465428 units of Food2_Egg,_white,_dried
0.45547635 units of Food2_Jew's_ear,_(pepeao),_dried
0.16709335 units of Food2_KRAFT,_Sugar_Free_COUNTRY_TIME_Pink_Lemonade_Mix,_with_vitamin_
0.17539672 units of Food2_Leavening_agents,_cream_of_tartar
0.27078841 units of Food2_Lettuce,_red_leaf,_raw
0.34185148 units of Food2_Margarine,_regular,_hard,_soybean_(hydrogenated)_and_cottonseed
0.64387872 units of Food2_Oil,_whale,_bowhead_(Alaska_Native)
9999.0273 units of Food2_Water,_bottled,_non_carbonated,_CALISTOGA
0.015495182 units of Food2_Whale,_beluga,_meat,_air_dried,_raw_(Alaska_Native)

Total cholesterol of food = 0.00
